In [1]:
import os
import pandas as pd
import numpy as np

from controllers.inference_controllers.sandbox import Sandbox
from controllers.inference_controllers.generator import Generator
from controllers.inference_controllers.optimiser import Optimiser

from toolboxes.plotting_toolbox.domain import Domain
from toolboxes.inference_toolbox.parameter import Parameter
from toolboxes.inference_toolbox.model import Model
from toolboxes.inference_toolbox.likelihood import Likelihood

current_directory = os.getcwd()
if current_directory != '/project/':
    os.chdir('/project/')

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [2]:
# data_params = {
#     'data_type': 'normalised_data',
#     'data_select': 'GBR_data',
#     'normaliser_select': 'GBR_normaliser',
#     'normaliser_params':{
#         'experiments_list': [
#             'Exp1',
#             'Exp2',
#             'Exp3',
#             'Exp4',
#             'Exp5',
#             'Exp6',
#             'Exp7',
#             'Exp8',
#             'Exp9',
#             'Exp10',
#             'Exp11',
#             'Exp12',
#         ],
#         'meta_data_select': 'GBR_data_summary',
#         'input_header': 'Concentration'
#     },
#     'log':True,
#     'output_header': 'Concentration',
#     'gridding': [100,100,25]
# }

data_params = {
    'data_type': 'simulated_data',
    'model': {
        'model_select': 'log_gpm_norm',
        'model_params':{
            'H': 10
        },
        'inference_params':{
            'I_y': 0.1,
            'I_z': 0.1,
            'Q': 3e13,
            'sigma': 1
        }
    },
    'domain':{
        'domain_select': 'cone_from_source_z_limited',
        'domain_params': {
            'r': 100,
            'theta': np.pi/8,
            'source': [0,0,10]
        },
        'resolution': 20
    },
    'noise_dist': 'gaussian',
    'noise_level': 1,
    'output_header': 'Concentration',
}

default_params = {
    'infered_params':pd.Series({
        'model_params':pd.Series({
            'I_y_and_I_z': Parameter('I_y','I_z', prior_select='log_norm')
                                    .add_prior_param('mu', [0.12,0.09])
                                    .add_prior_param('cov', [[0.01,0],[0,0.01]]),
                                    'Q': Parameter('Q', prior_select = 'log_norm').add_prior_param('mu', 3.5e13).add_prior_param('sigma',1e7),
        }),
        'likelihood_params':pd.Series({
            'sigma': Parameter('sigma', prior_select = 'log_norm').add_prior_param('mu', 1.25).add_prior_param('sigma',0.7)
        })
    }),
    'model':Model('log_gpm_norm').add_model_param('H',10),
    'likelihood': Likelihood('gaussian'),
    'sampler': {
        'n_samples': 10001,
        'n_chains': 3,
        'thinning_rate': 1
    }
}

# default_params = {
#     'infered_params':pd.Series({
#         'model_params':pd.Series({
#             'I_y': Parameter('I_y', prior_select='log_norm').add_prior_param('mu', 1).add_prior_param('sigma',1),
#             'I_z': Parameter('I_z', prior_select='log_norm').add_prior_param('mu', 1).add_prior_param('sigma',1),
#             'Q': Parameter('Q', prior_select = 'log_norm').add_prior_param('mu', 3.5e13).add_prior_param('sigma',1e7),
#         }),
#         'likelihood_params':pd.Series({
#             'sigma': Parameter('sigma', prior_select = 'log_norm').add_prior_param('mu', 1.25).add_prior_param('sigma',0.7)
#         })
#     }),
#     'model':Model('log_gpm_norm').add_model_param('H',10),
#     'likelihood': Likelihood('gaussian'),
#     'sampler': {
#         'n_samples': 1000,
#         'n_chains': 3,
#         'thinning_rate': 1
#     }
# }

prior_plots = [
    {
        'I_y': [0.0001, 1.7],
        'references':{
            'I_y' : [0.22, 1.6, 0.11, 0.08, 0.06, 0.04],
            'labels': ['A','B','C','D','E','F'] 
        }
    },
    {
        'I_z': [0.0001, 1.7],
        'references':{
            'I_z' : [0.2, 1.2, 0.08, 0.06, 0.03, 0.016],
            'labels': ['A','B','C','D','E','F'] 
        }
    },
    {
        'Q': [1e12, 5e13],
        'references':{
            'Q' : [2.82e13, 3.11e13, 2.89e13, 4.83e13],
            'labels': ['250m','200m','750m','1000m'] 
        }
    },
    {
        'sigma': [0.001, 4],
    },
    {
        'I_y': [0.0001, 1.7],
        'I_z': [0.0001, 1.7],
        'references':{
            'I_y' : [0.22, 1.6, 0.11, 0.08, 0.06, 0.04],
            'I_z' : [0.2, 1.2, 0.08, 0.06, 0.03, 0.016],
            'labels': ['A','B','C','D','E','F'] 
        }
    }
]


results_name = 'testing_dists2'

sandbox = Sandbox(results_name=results_name, 
                  data_params=data_params,
                  default_params=default_params)

visualiser = sandbox.run()
# visualiser.get_summary()
visualiser.get_traceplot()
# visualiser.get_autocorrelations()
visualiser.get_prior_plots(prior_plots)

# domain = Domain('cone_from_source_z_limited', resolution=80)
# domain.add_domain_param('r', 1000)
# domain.add_domain_param('theta', np.pi/8)
# domain.add_domain_param('source', [0,0,10])

# visualiser.visualise_results(domain = domain, name = 'small_scale_3D_plots', title='Log Concentration of Droplets', log_results=False)
# visualiser.animate(name = 'small_scale_3D_plots')

Loading Chain Samples...
Loading Samples...
Traceplot 1 already exists
Traceplot 2 already exists
Traceplot 3 already exists


In [3]:
analysis_iterations = {
    'parameters_1':
    [
        'I_y_and_I_z_mu_0'
    ],
    'parameters_2':
    [
        'I_y_and_I_z_mu_1'

    ],
    'values_1':
    [
        np.array([1, 1.1])     
    ],

    'values_2':
    [
        np.array([1, 1.1]) 
    ]
}

generator = Generator(results_name=results_name, 
                  data_params=data_params,
                  default_params=default_params)

for i in range(len(analysis_iterations['parameters_1'])):
    parameter_1 = analysis_iterations['parameters_1'][i]
    parameter_2 = analysis_iterations['parameters_2'][i]

    print('Working on varying ' + parameter_1 + '...')
    values_1 = analysis_iterations['values_1'][i]
    values_2 = analysis_iterations['values_2'][i]
    inputs = generator.vary_two_parameters(parameter_1, parameter_2, values_1, values_2, scale_1='linear', scale_2='linear', plot=True)

Working on varying I_y_and_I_z_mu_0...
Generating instance 1...
Generating instance 2...


sample: 100%|██████████| 12501/12501 [00:02<00:00, 5102.72it/s, 3 steps of size 6.34e-01. acc. prob=0.89]


Generating instance 3...


sample: 100%|██████████| 12501/12501 [00:02<00:00, 5323.22it/s, 7 steps of size 5.56e-01. acc. prob=0.92]


Generating instance 4...


sample: 100%|██████████| 12501/12501 [00:02<00:00, 5388.03it/s, 3 steps of size 6.06e-01. acc. prob=0.91]
